In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
gene_x_sample = pd.read_table(
    '../output/max_tpm_log__gene_x_sample.tsv',
    index_col=0,
)

In [ ]:
target_x_sample = pd.read_table(
    '../data/target_x_sample.tsv',
    index_col=0,
)

target_x_sample

In [ ]:
gene_set_directory_path = '../data/gene_set'

gene_set_file_paths = tuple(os.path.join(
    gene_set_directory_path,
    gene_set_file_name,
) for gene_set_file_name in os.listdir(gene_set_directory_path) if gene_set_file_name.endswith('.gmt'))

gene_set_file_paths

In [ ]:
normalization_method = None

statistic = 'auc'

In [ ]:
for gene_set_file_path in gene_set_file_paths:

    print(gene_set_file_path)

    gene_sets = ccal.read_gmt(gene_set_file_path)

    gene_set_file_name = os.path.split(gene_set_file_path)[1]

    output_directory_path = '../output/gene_set/{}'.format(gene_set_file_name)

    gene_set_x_sample_file_path = '{}/gene_set_x_sample.tsv'.format(output_directory_path)

    if os.path.isfile(gene_set_x_sample_file_path):

        gene_set_x_sample = pd.read_table(
            gene_set_x_sample_file_path,
            index_col=0,
        )

    else:

        gene_set_x_sample = ccal.single_sample_gseas(
            gene_x_sample.dropna(),
            gene_sets,
            normalization_method=normalization_method,
            statistic=statistic,
            file_path=gene_set_x_sample_file_path,
        )

    make_match_panels(
        target_x_sample,
        gene_set_x_sample,
        N_JOB,
        EXTREME_FEATURE_THRESHOLD,
        N_SAMPLING,
        N_PERMUTATION,
        '({}) '.format(gene_set_file_name),
        output_directory_path,
        reset=True,
    )

In [ ]:
gene_sets_to_peek_output_directory_path = '../output/gene_set/peek'

ccal.establish_path(
    gene_sets_to_peek_output_directory_path,
    'directory',
)

In [ ]:
all_gene_sets = ccal.drop_df_slice(
    ccal.read_gmts(gene_set_file_paths),
    1,
    min_n_not_na_unique_value=3,
)

all_gene_sets

In [ ]:
gene_sets_to_peek = np.full(
    all_gene_sets.shape[0],
    False,
)

for gene_set in GENE_SETS_TO_PEEK:

    gene_sets_to_peek |= all_gene_sets.index.str.lower() == gene_set.lower()

for gene_set_keyword in GENE_SET_KEYWORDS_TO_PEEK:

    gene_sets_to_peek |= all_gene_sets.index.str.lower().str.contains(gene_set_keyword)

gene_sets_to_peek = all_gene_sets.loc[gene_sets_to_peek]

all_genes_from_gene_sets_to_peek = gene_sets_to_peek.unstack().dropna().unique()

all_genes_from_gene_sets_to_peek = pd.Series(
    all_genes_from_gene_sets_to_peek,
    name='All Genes from Gene Sets to Peek',
    index=('Gene {}'.format(i) for i in range(all_genes_from_gene_sets_to_peek.size)),
)

gene_sets_to_peek = gene_sets_to_peek.append(all_genes_from_gene_sets_to_peek)

gene_sets_to_peek

In [ ]:
gene_set_x_sample_file_path = '{}/gene_set_x_sample.tsv'.format(gene_sets_to_peek_output_directory_path)

gene_set_x_sample = ccal.single_sample_gseas(
    gene_x_sample.dropna(),
    gene_sets_to_peek,
    normalization_method=normalization_method,
    statistic=statistic,
    file_path=gene_set_x_sample_file_path,
)

make_match_panels(
    target_x_sample,
    gene_set_x_sample,
    N_JOB,
    None,
    N_SAMPLING,
    N_PERMUTATION,
    '(gene sets to peek) ',
    gene_sets_to_peek_output_directory_path,
    reset=True,
)

In [ ]:
for target_name, target in target_x_sample.iterrows():

    target = target[target != -1]

    print(target)

    gene_set_score_p_value_fdr = ccal.gsea(
        gene_x_sample[target.index].dropna(),
        target,
        gene_sets_to_peek,
        method=ccal.compute_information_coefficient,
        normalization_method=None,
        statistic=statistic,
        plot=True,
        directory_path='{}/gsea/{}'.format(
            gene_sets_to_peek_output_directory_path,
            target_name,
        ),
    )

    scores = gene_set_score_p_value_fdr['Score']

    print('{}: N={} & mean={:.3f} & STD={:.3f}'.format(
        target_name,
        scores.shape[0],
        scores.mean(),
        scores.std(),
    ))